<a href="https://colab.research.google.com/github/scartt/CS505_PA/blob/main/CS505_PA3_Sijia_Li.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
# Sijia Li
# CS505 PA3
# scartt@bu.edu
# Hi, there are some problems with my initial scraping, so I only write the functions in some tasks without load the data. 

## Task 1 Scape and import the data

In [86]:
#import libraries
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time # for setting up a delay on getting htmls from wiki server.
from tqdm import tqdm
import numpy as np
import pandas as pd
import sys
import csv

# First, get the page info from wiki server given an URL.
def getPageFromWiki(url):
  html = urlopen(url) 
  soup = BeautifulSoup(html, 'html.parser')
  return soup

# Second, get the title of the wiki page
def getHeading(soup):
  heading = ""
  heading = soup.title
  return heading

# Third, get the article part of the wiki page 
def getContent(page):
  content = ""
  content = page.get_text()
  return content

# Fourth, get the links that the article part mentioned and specifically, linking to other wiki pages.
def getLinks(page):
  linksDict = {}
  for link in page.findAll('a'):
    linksDict[link.title]=link.get('href')
  return linksDict

In [87]:
# Once you've implemented the above functions, run the following piece to see if a dictionary that contains the wiki articles we scraped.
# Run a for loop to get all the article contents from Wikipedia.
pageDict = {}

page = getPageFromWiki('https://en.wikipedia.org/wiki/Fishing') # scrap the main page we want. 
header = getHeading(page)
content = getContent(page)
pageDict[header] = content
print(pageDict)

linksDict = getLinks(page) # get the links contained in the article part of the page.
print("a set of {} links are found.".format(len(linksDict)))

for title in tqdm(list(linksDict.keys())): # set up a loop to , set a delay at each iteration
  url = linksDict[title]
  page = getPageFromWiki(url)
  header = getHeading(page)
  content = getContent(page)
  pageDict[header] = content
  time.sleep(1) # Remember to set a delay >=1 second so you won't break the server.

print("a size of {} content dictionary is built.".format(len(pageDict)))

{<title>Fishing - Wikipedia</title>: '\n\n\n\nFishing - Wikipedia\ndocument.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b972d2dc-3c56-4be1-a221-ccea1c987041","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Fishing","wgTitle":"Fishing","wgCurRevisionId":1113942214,"wgRevisionId":1113942214,"wgArticleId":36581,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with dead external links","Articles with dead external links from October 2017","Articles with permanently dead external links","Articles with dead external links from December 2020","Webarchive template wayback links","Articles w

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]

a size of 2 content dictionary is built.


In [88]:
driveFolderDirectory = '/content/drive/MyDrive/Colab Notebooks/'
savedFileName = 'wikiContents.csv'
pathToSave = driveFolderDirectory + savedFileName

with open(pathToSave, 'w', newline='') as csvfile:
  fieldnames = ['idx','wikiTitle', 'wikiContents']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()
  for i,wikiContentKey in enumerate(pageDict.keys()):
    writer.writerow({'idx': i, 'wikiTitle': wikiContentKey,'wikiContents': pageDict[wikiContentKey]})

In [90]:
# data_fishing

## Task 2 Data Processing

In [91]:
# install spacy and related package(s)

# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

In [92]:
doc = data_fishing

In [118]:
import nltk
from nltk.stem import WordNetLemmatizer  
import string
import spacy
nlp = spacy.load('en_core_web_sm')

def preprocess(wikiTextDict):
  # Input: a wiki text dictionary with keys are titles and values are the corresponding texts.
  # Output: a wiki text dictionary with keys are the titles and the values are the preprocessed texts 
  # (sentences - tokens).
  doc = wikiTextDict.values
  # sub-task 1: remove all the references texts "[...]"
  for value in wikiTextDict:
    re.sub(r'\((?:[\w \.&]+\, )+[0-9]{4}\)', '', value)
    # sub-task 2: segment all the sentences in the wiki texts.
    nlp = spacy.load("en_core_web_sm")
    lemmatizer = nlp.get_pipe("lemmatizer")
    value = nlp(value)
    value = [sent.text for sent in value.sents]
    # sub-task 3: tokenize the sentences from sub-task 2.
    value = [token for token in value]
    # sub-task 4: lemmatize the tokens from sub-task 3.
    # value = [[token.text,token.lemma_] for token in value]
    # value = [token.lemma_ for token in value]
    # sub-task 5: lower-case the tokens from sub-task 3/4.
    value = [token.lower() for token in value]

  # You don't need to follow the order of the sub-tasks.

  pass

# Preprocess your data here.
preprocess(doc)

## Task 3 Word Types

In [121]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
import nltk
from nltk.stem import WordNetLemmatizer  
def computeFreq(wikiTextDict):

  # Input: a wiki text dictionary with keys are titles and values are the preprocessed corresponding texts.
  # Output: a dictionary with keys are the word types, and the values are the appearance counts of the word types
  word_frequencies = nltk.FreqDist(wikiTextDict)
  sorted_counts = sorted(word_frequencies.items() , key = lambda x: x[1] ,reverse = True)
  return sorted_counts
  pass

  # Compute the frequency dictionary here.
articles = [article for article in data_fishing if article != '']
results = [computeFreq(article) for article in articles]
results

[[('i', 7),
  ('t', 4),
  ('e', 3),
  ('<', 2),
  ('l', 2),
  ('>', 2),
  (' ', 2),
  ('F', 1),
  ('s', 1),
  ('h', 1),
  ('n', 1),
  ('g', 1),
  ('-', 1),
  ('W', 1),
  ('k', 1),
  ('p', 1),
  ('d', 1),
  ('a', 1),
  ('/', 1)],
 [('i', 5),
  ('t', 4),
  ('e', 3),
  ('<', 2),
  ('l', 2),
  ('>', 2),
  ('M', 1),
  ('d', 1),
  ('a', 1),
  ('W', 1),
  ('k', 1),
  ('/', 1)]]

## Task 4 Top 20 Words

In [97]:
import collections
from collections import Counter

In [98]:
# results.most_common(20)

## Task 5 Word Cloud

In [99]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [100]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

def plotWordCloud(image):
  # Input: word cloud image
  # Output: (display the cloud image in the output)
  
  pass

def generateWordCloud(text):
  # Input: all texts in the scraped wiki data.
  # Output: word cloud image.
  wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2).generate(text)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.show()
  pass

# Draw your word cloud here


## Task 6 Tweet Revisited

In [102]:
def loadTweetTextFromCSV(csvPath):
  tweetDict = {}
  with open(csvPath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      tweetDict[int(row['idx'])] = row['tweetText']
  return tweetDict

# processedTweetData = preprocess(tweetDict) # here we assume you have implemented the preprocess function in task 2.
fish = loadTweetTextFromCSV('/content/drive/My Drive/tweets_fish.csv')

## Task 7

In [103]:
def computeOOVWordTypes(tweetVocabDict, wikiVocabDict):

  # Input: a dictionary of tweet data vocabulary, a dictionary of wiki data vocabulary.
  # Output: the ratio of word types in your tweets that are out-of-vocabulary w.r.t. wiki vocabulary
  # v.s. total number of word types in your tweet data.

  # The ratio should be in percentage.

  pass

# Print your ratio here.

## Task  8

In [104]:
def computeOOVWordTokens(tweetVocabDict, wikiVocabDict):

  # Input: a dictionary of tweet data vocabulary, a dictionary of wiki data vocabulary. (E.g. computed from task 3)
  # Output: the ratio of word tokens in your tweets that are out-of-vocabulary w.r.t. wiki vocabulary
  # v.s. total number of word tokens in your tweet data.

  # Remeber this time we count the number of tokens instead of types. The ratio should be in percentage.
  
  pass

# Print your ratio here.

## Task 9

## Task 10

## Task 11 News Scraping

In [106]:
#install newspaper
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=09695e7f93f2af2ae39abfa7ead0957c891fb40b2cbfd91095bc57b872c09f87
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=2044c5a62590e25d5b92d34c2861e07701e595119a0c7dd5374b4

In [130]:
fox_text = []
with open('fox_news.jsonl') as f:
    for line in f:
        js = json.loads(line)
        fox_text.append(js['bodytext'])
print("Number of Fox news artivle: {}".format(len(fox_text)))

FileNotFoundError: ignored

In [108]:
from newspaper import Article
from tqdm import tqdm

def getNewsDict(url_list):

  # Your key should be the news title and value should be the article text of the news.
  newsDict = {}
  # IMPLEMENT YOUR CODE HERE:# 
  
  return newsDict


abcNews = getNewsDict(abcUrlList)
foxNews = getNewsDict(foxUrlList)

NameError: ignored

In [105]:
# Here is a function you could load the text data if your saved data follows
# the format we provide the in first lab section code.

def loadNewsTexts(csvPath):

  # the function returns two dictionaries, one for ABC news text data and one for Fox news text data

  abcNewsRawTextDict = {}
  foxNewsRawTextDict = {}
  with open(csvPath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      if (row['newsSource'] == "ABCNews"):
        abcNewsRawTextDict[row['newsTitle']] = row['newsContents']
      else:
        foxNewsRawTextDict[row['newsTitle']] = row['newsContents']

  return abcNewsRawTextDict,foxNewsRawTextDict

# Load your news text data here
# abcNewsDict,foxNewsDict = loadNewsTexts('./newsContents.csv')

In [ ]:
abc_words = []
for text in abc_sentences:
    for word in text:
        if (word!='pm' and word!='nt'):
            abc_words.append(word)

In [ ]:
fox_words = []
for text in fox_sentences:
    for word in text:
        fox_words.append(word)

In [ ]:
make_wordcloud(" ".join(abc_words), 'abc_news.png')

In [ ]:
make_wordcloud(" ".join(fox_words), 'fox_news.png')